In [4]:
import requests
import pandas as pd
from io import BytesIO

lsts_url = "https://github.com/DJRT-11/MINE4101-T1/blob/main/data/listings.csv.gz"
cldr_url = "https://github.com/DJRT-11/MINE4101-T1/blob/main/data/calendar.csv.gz"
revs_url = "https://github.com/DJRT-11/MINE4101-T1/blob/main/data/reviews.csv.gz"

lsts_rsp = requests.get(lsts_url,params={'raw':'true'})
cldr_rsp = requests.get(cldr_url,params={'raw':'true'})
revs_rsp = requests.get(revs_url,params={'raw':'true'})

lsts_fpt = BytesIO(lsts_rsp.content)
cldr_fpt = BytesIO(cldr_rsp.content)
revs_fpt = BytesIO(revs_rsp.content)

lsts_df = pd.read_csv(lsts_fpt,compression='gzip')
cldr_df = pd.read_csv(cldr_fpt,compression='gzip')
revs_df = pd.read_csv(revs_fpt,compression='gzip')

In [8]:
lsts_df.head(10)

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,154221,https://www.airbnb.com/rooms/154221,20230608145305,2023-06-10,previous scrape,Rental unit in London · 2 bedrooms · 2 beds · ...,"hi, i am renting my 2 bedroom flat in Pimlico ...","a very save area of central London , Pimlico i...",https://a0.muscache.com/pictures/3510524/72655...,211018,...,5.00,5.00,5.00,NaN,f,1,1,0,0,0.01
1,157714,https://www.airbnb.com/rooms/157714,20230608145305,2023-06-09,city scrape,Townhouse in Hammersmith · ★4.71 · 1 bedroom ·...,"Large, light and spacious loft bedroom with a ...",The neighbourhood is extremely convenient for ...,https://a0.muscache.com/pictures/4129744/10db5...,757377,...,4.83,4.76,4.66,NaN,f,1,0,1,0,1.21
2,13913,https://www.airbnb.com/rooms/13913,20230608145305,2023-06-09,city scrape,Rental unit in Islington · ★4.80 · 1 bedroom ·...,My bright double bedroom with a large window h...,Finsbury Park is a friendly melting pot commun...,https://a0.muscache.com/pictures/miso/Hosting-...,54730,...,4.82,4.69,4.69,NaN,f,2,1,1,0,0.26
3,15400,https://www.airbnb.com/rooms/15400,20230608145305,2023-06-09,city scrape,Rental unit in London · ★4.80 · 1 bedroom · 1 ...,Lots of windows and light. St Luke's Gardens ...,It is Chelsea.,https://a0.muscache.com/pictures/428392/462d26...,60302,...,4.83,4.93,4.74,NaN,f,1,1,0,0,0.57
4,306333,https://www.airbnb.com/rooms/306333,20230608145305,2023-06-09,city scrape,Rental unit in London · ★4.32 · 2 bedrooms · 3...,Welcome to my lovely 2-bedroom flat in Hoxton!...,"In Shoreditch, empty walls are future murals. ...",https://a0.muscache.com/pictures/32c818d1-7762...,1573870,...,4.84,4.89,4.53,NaN,f,1,1,0,0,0.25
5,159736,https://www.airbnb.com/rooms/159736,20230608145305,2023-06-09,city scrape,Rental unit in Lambeth · ★4.72 · 1 bedroom · 1...,Calm sunny double room with a queen size bed a...,We love that in Loughborough Junction we live ...,https://a0.muscache.com/pictures/1067303/d2300...,766056,...,4.89,4.34,4.66,NaN,f,4,0,4,0,0.66
6,306405,https://www.airbnb.com/rooms/306405,20230608145305,2023-06-08,city scrape,Rental unit in Dagenham · ★4.0 · 1 bedroom · 1...,<b>The space</b><br />It is suitable for solo ...,NaN,https://a0.muscache.com/pictures/4070359/6d80d...,1575800,...,3.00,3.00,3.67,NaN,f,1,0,1,0,0.02
7,306677,https://www.airbnb.com/rooms/306677,20230608145305,2023-06-10,previous scrape,Rental unit in London · ★4.56 · 2 bedrooms · 3...,Non smoking apartment worth uk£600k (US$900k) ...,Fantastic location to access all tourist attra...,https://a0.muscache.com/pictures/3419083/e5533...,1577254,...,4.78,4.41,4.55,NaN,t,6,6,0,0,1.58
8,17402,https://www.airbnb.com/rooms/17402,20230608145305,2023-06-09,city scrape,Rental unit in London · ★4.76 · 3 bedrooms · 3...,You'll have a wonderful stay in this superb mo...,"Location, location, location! You won't find b...",https://a0.muscache.com/pictures/39d5309d-fba7...,67564,...,4.71,4.88,4.60,NaN,f,9,9,0,0,0.36
9,165336,https://www.airbnb.com/rooms/165336,20230608145305,2023-06-10,previous scrape,Rental unit in Kensington · ★4.75 · 1 bedroom ...,A stylish apartment close to Portobello market...,"Notting Hill has many cafes, bars and restaura...",https://a0.muscache.com/pictures/60757460/47f8...,761400,...,4.96,4.87,4.72,NaN,f,1,1,0,0,1.53


In [6]:
cldr_df.head(10)

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,13913,2023-06-09,f,$79.00,$79.00,1.0,29.0
1,13913,2023-06-10,f,$79.00,$79.00,1.0,29.0
2,13913,2023-06-11,f,$79.00,$79.00,1.0,29.0
3,13913,2023-06-12,t,$79.00,$79.00,1.0,29.0
4,13913,2023-06-13,t,$79.00,$79.00,1.0,29.0
5,13913,2023-06-14,t,$79.00,$79.00,1.0,29.0
6,13913,2023-06-15,t,$79.00,$79.00,1.0,29.0
7,13913,2023-06-16,t,$79.00,$79.00,1.0,29.0
8,13913,2023-06-17,f,$79.00,$79.00,1.0,29.0
9,13913,2023-06-18,t,$79.00,$79.00,1.0,29.0


In [7]:
revs_df.head(10)

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,306333,121679704,2016-12-18,8698767,Ben,"Great find, and great value! <br/>Right in th..."
1,154221,59561487,2016-01-12,1655084,Daria,Small and cosy flat with all goods you need in...
2,306333,122121197,2016-12-21,72253091,Matthew,A quiet house in the Hoxton area. Felt very ho...
3,306333,123425430,2016-12-29,18523201,Kom,"We had a great stay in London, and Rob's flat ..."
4,306333,126479992,2017-01-12,9940848,Marisa,This flat is perfect if you are interested in ...
5,306333,126951378,2017-01-15,38360161,Isaac,Great location
6,306333,129056149,2017-01-29,45678790,Cathal,Rob's place is in a great location for friends...
7,306333,130060551,2017-02-04,4737225,Andrew,The flat is a rare find for this part of Londo...
8,306333,130653008,2017-02-07,11272550,Travis,"Great place, great location, great bang for bu..."
9,306333,131398648,2017-02-12,17487017,Myriam & Julien,You can provide better advice to find the flat...
